### LangChain + Deep Lake + ChatGPT-3.5 快速学习github开源代码 

#### 1. dependences
##### 1.1 pinecone - online vectors DB (https://www.pinecone.io/)
##### 1.2 tiktoken - convert the text to vectors

In [11]:
#%pip install --upgrade langchain pinecone-client openai tiktoken


In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore")
#os.environ.clear()
load_dotenv()
api_key = os.environ['OPENAI_API_KEY']

In [4]:

embeddings = OpenAIEmbeddings(disallowed_special=())

In [5]:
from langchain.document_loaders import TextLoader

root_dir = '../../../open-interpreter/interpreter'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try:
            loader = TextLoader(os.path.join(dirpath, file),encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e:
            pass


In [6]:
len(docs)

27

In [7]:
from langchain.text_splitter import CharacterTextSplitter
# 对文本切块，4000的字符切一次，每个切块的上下文件有100个字符的重叠
text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=50)
texts= text_splitter.split_documents(docs)
len(texts)

43

In [14]:
from langchain.vectorstores import Pinecone
import pinecone


index_name = "interpreter-openai-embeddings"

# initialize pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)



# First, check if our index already exists. If it doesn't, we create it
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
      name=index_name,
      metric='cosine',
      dimension=1536  
)

In [18]:
# The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
docsearch = Pinecone.from_documents(documents=texts, embedding=embeddings, index_name=index_name)

In [19]:
# if you already have an index, you can load it like this
# docsearch = Pinecone.from_existing_index(index_name, embeddings)
query = "open-interpreter调用了openai的哪些接口?"
docs = docsearch.similarity_search(query)
print(docs)

[Document(page_content="You are Open Interpreter, a world-class programmer that can complete any goal by executing code.\nFirst, write a plan. **Always recap the plan between each code block** (you have extreme short-term memory loss, so you need to recap the plan between each message block to retain it).\nWhen you send a message containing code to run_code, it will be executed **on the user's machine**. The user has given you **full and complete permission** to execute any code necessary to complete the task. You have full access to control their computer to help them. Code entered into run_code will be executed **in the users local environment**.\nNever use (!) when running commands.\nOnly use the function you have been provided with, run_code.\nIf you want to send data between programming languages, save the data to a txt or json.\nYou can access the internet. Run **any code** to achieve the goal, and if at first you don't succeed, try again and again.\nIf you receive any instructio

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model="gpt-3.5-turbo-16k",openai_api_key=api_key,temperature=0.1)
qa = ConversationalRetrievalChain.from_llm(model, retriever=docsearch.as_retriever())

In [22]:
questions = [
    """
    请为我提供有关open-interpreter项目的信息。
    我已经在本地安装了该项目的代码，并希望了解open-interpreter全部的工作流程。
    """
]
chat_history = []

for question in questions:
    result = qa({"question":question,"chat_history":chat_history})
    chat_history.append((question,result['answer']))
    print(f"Q:\n {question} \n")
    print(f"A:\n {result['answer']} \n")

Q:
 
    请为我提供有关open-interpreter项目的信息。
    我已经在本地安装了该项目的代码，并希望了解open-interpreter全部的工作流程。
     

A:
 open-interpreter项目是一个开源项目，旨在为开发者提供一个交互式的编程环境。它允许用户在本地运行LLMs（Code-Llama），并提供了一些安全功能，如Eric Allen的`--scan`模式安全措施，由GuardDog和Semgrep提供支持。此外，open-interpreter还提供了一个桌面应用程序，用户可以通过提前注册获得早期访问权限。该项目的工作流程包括在本地运行LLMs、使用CLI和Python进行交互，并支持恢复聊天记录。如果你已经在本地安装了open-interpreter的代码，你可以通过运行相应的命令来体验它的功能。 



In [23]:
def ask(question,chat_history):
    res = qa({"question":question,"chat_history":chat_history})
    print(f"Q:\n {question} \n")
    print(f"A:\n {res['answer']} \n")

In [24]:
ask("open-interpreter在执行任务的过程中,调用了openai的哪些接口?",chat_history)

Q:
 open-interpreter在执行任务的过程中,调用了openai的哪些接口? 

A:
 Open Interpreter 在执行任务的过程中调用了以下 OpenAI 的接口：

1. OpenAI GPT-3：用于生成自然语言文本，包括解释代码、回答问题等。
2. OpenAI GPT-4：用于生成自然语言文本，功能更强大。
3. OpenAI Code-Llama：用于执行代码块，返回执行结果。
4. Azure OpenAI Service：用于执行代码块，返回执行结果。

这些接口提供了强大的功能，使 Open Interpreter 能够执行各种任务，并为用户提供帮助。 



In [25]:
ask("open-interpreter调用OpenAI的接口可以具体列举一下吗?",chat_history)

Q:
 open-interpreter调用OpenAI的接口可以具体列举一下吗? 

A:
 Open Interpreter调用OpenAI的接口主要有以下功能：

1. 使用GPT-4进行自然语言处理和生成。可以使用GPT-4来提供智能的文本回复、语言翻译、文本摘要等功能。

2. 使用Code-Llama进行代码解释和执行。可以使用Code-Llama来解释和执行多种编程语言的代码，包括Python、R、Shell、Applescript、JavaScript和HTML。

3. 使用Azure OpenAI Service API进行代码解释和执行。可以使用Azure OpenAI Service API来解释和执行多种编程语言的代码，包括Python、R、Shell、Applescript、JavaScript和HTML。

这些功能可以帮助用户在编程和自然语言处理方面提供强大的支持和帮助。 



In [26]:
ask("在使用Code-Llama进行代码解释和执行,open-interpreter是怎么做到的?",chat_history)

Q:
 在使用Code-Llama进行代码解释和执行,open-interpreter是怎么做到的? 

A:
 根据提供的上下文，Open Interpreter正在使用Code-Llama来支持运行LLMs（Language Learning Models）并进行代码解释和执行。Code-Llama是Eric Allen开发的工具，它具有安全性措施，由GuardDog和Semgrep提供支持。它还提供了与Python相同的命令行界面（CLI）功能，包括恢复聊天的功能。此外，Open Interpreter还计划开发一个桌面应用程序，以提供更便捷的访问方式。 

